In [1]:
import settings
import helpers
import sys
import os
import glob
import random
import pandas
import ntpath
import cv2
import numpy as np
from tensorflow.keras import backend as K
import math
import pydicom as dicom
from PIL import Image, ImageDraw

Computer:  DESKTOP-5Q4EG11


In [2]:
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
import Train
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.compat.v1.Session(config=config))

In [3]:
pred_labels = pandas.read_csv('F:/Cengiz/Nodules-Detection/prediction/predictions10_luna16_fs/1.3.6.1.4.1.14519.5.2.1.6279.6001.178391668569567816549737454720.csv')
pred_labels.head()

,anno_index,coord_x,coord_y,coord_z,nodule_chance,status
0,0,0.2316,0.3579,0.5181,0.9527,NaN
1,1,0.2632,0.3579,0.5181,0.9764,NaN
2,2,0.2316,0.3895,0.5181,0.8219,NaN
3,3,0.2632,0.3895,0.5181,0.9233,NaN
4,4,0.2316,0.3579,0.5542,0.9124,NaN


In [10]:
def load_patient(src_dir):
    # slices = [dicom.read_file(src_dir + '/' + s) for s in os.listdir(src_dir)]
    slices = []
    print(src_dir)
    for files in glob.glob(src_dir + "/*.dcm"):
        slices.append(dicom.read_file(files))
    print(len(slices))
    slices.sort(key=lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    if slice_thickness == 0:
        slice_thickness = np.abs(slices[4].ImagePositionPatient[2] - slices[5].ImagePositionPatient[2])
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(float)
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number]
        image[slice_number] += intercept

    return np.array(image, dtype=np.float64)


In [16]:
INPUT_DICOM_PATH = 'D:/LIDC-IDRI/LIDC-IDRI-0070/1.3.6.1.4.1.14519.5.2.1.6279.6001.101493103577576219860121359500/1.3.6.1.4.1.14519.5.2.1.6279.6001.178391668569567816549737454720'
slices = load_patient(INPUT_DICOM_PATH)
pixels = get_pixels_hu(slices)
image = pixels
shape_image = image.shape
nodules_coord = []
slices_nod_num = []
for index, row in pred_labels.iterrows():
    coord_z = row['coord_z'] * shape_image[0]
    coord_z = int(round(coord_z))
    coord_y = row['coord_y'] * shape_image[2]
    coord_y = int(round(coord_y))
    coord_x = row['coord_x'] * shape_image[1]
    coord_x = int(round(coord_x))
    nodules_coord.append([coord_x,coord_y,coord_z])
    slices_nod_num.append(coord_z)
print(nodules_coord)

D:/LIDC-IDRI/LIDC-IDRI-0070/1.3.6.1.4.1.14519.5.2.1.6279.6001.101493103577576219860121359500/1.3.6.1.4.1.14519.5.2.1.6279.6001.178391668569567816549737454720
133
[[119, 183, 69], [135, 183, 69], [119, 199, 69], [135, 199, 69], [119, 183, 74], [135, 183, 74], [119, 199, 74], [135, 199, 74], [216, 183, 79], [232, 183, 79], [377, 216, 79], [361, 232, 79], [377, 232, 79], [393, 313, 79], [216, 183, 83], [232, 183, 83], [183, 199, 83], [216, 199, 83], [232, 199, 83], [183, 216, 83], [377, 216, 83], [361, 232, 83], [377, 232, 83], [183, 199, 88], [183, 216, 88], [345, 313, 88], [329, 329, 88], [345, 329, 88], [345, 313, 93], [329, 329, 93], [345, 329, 93], [167, 313, 112], [167, 329, 112], [167, 313, 117], [183, 313, 117], [167, 329, 117], [183, 329, 117]]


In [27]:
slices_org_num = []
nodules_coord = []
patient_id = '1.3.6.1.4.1.14519.5.2.1.6279.6001.178391668569567816549737454720'
patient_img = helpers.load_patient_images(patient_id, settings.LIDC_EXTRACTED_IMAGE_DIR, "*_i.png", [])
print(patient_img.shape)
for index, row in pred_labels.iterrows():
    p_x,p_y,slice_num = helpers.percentage_to_pixels(row['coord_x'], row['coord_y'], row['coord_z'], patient_img)
    nodules_coord.append([p_x,p_y,slice_num])
print(nodules_coord)

(332, 380, 380)
[[88, 136, 172], [100, 136, 172], [88, 148, 172], [100, 148, 172], [88, 136, 184], [100, 136, 184], [88, 148, 184], [100, 148, 184], [160, 136, 196], [172, 136, 196], [280, 160, 196], [268, 172, 196], [280, 172, 196], [292, 232, 196], [160, 136, 208], [172, 136, 208], [136, 148, 208], [160, 148, 208], [172, 148, 208], [136, 160, 208], [280, 160, 208], [268, 172, 208], [280, 172, 208], [136, 148, 220], [136, 160, 220], [256, 232, 220], [244, 244, 220], [256, 244, 220], [256, 232, 232], [244, 244, 232], [256, 244, 232], [124, 232, 280], [124, 244, 280], [124, 232, 292], [136, 232, 292], [124, 244, 292], [136, 244, 292]]


In [21]:
print(shape_image)

(133, 512, 512)


In [18]:
def apply_ct_window(img, window):
    # window = (window width, window level)
    R = (img-window[1]+0.5*window[0])/window[0]
    R[R<0] = 0
    R[R>1] = 1
    return R
def normalize(image):
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image > 1] = 1.
    image[image < 0] = 0.
    return image
#img = image[0]
#print(img.shape)
#display_img = apply_ct_window(img, [400,-1000])
#display_img = normalize(img)
#ds = slices[0]

In [19]:
for count,slice_ in enumerate(slices):
    #display_img = normalize(img)
    pixels = get_pixels_hu([slice_])
    image = pixels
    img = image[0]
    #print(img.shape)
    ds = slice_
    normalized_image = normalize(img)
    if count in slices_nod_num:
        for coord in nodules_coord:
            if count == coord[2]:
                top, left, bottom, right = [coord[1]-15,coord[0]-15,coord[1]+15,coord[0]+15]
                #top, left, bottom, right = [211,99,291,158]
                thickness = 2
                img_bbox = Image.fromarray((255*normalized_image).astype('uint8'))
                img_bbox = img_bbox.convert('RGB')
                #rect = Image.new ('RGB', img_bbox.size)
                draw = ImageDraw.Draw(img_bbox)
                for i in range(thickness):
                    draw.rectangle(
                        [left + i, top + i, right - i, bottom - i],
                        outline=(255,0,0,125) #transperency 125 (1255)
                    )
                del draw
                #img_bbox = Image.alpha_composite(img_bbox,rect)
                img_bbox = np.asarray(img_bbox)

                # modify DICOM tags
                ds.PhotometricInterpretation = 'RGB'
                ds.SamplesPerPixel = 3
                ds.BitsAllocated = 8
                ds.BitsStored = 8
                ds.HighBit = 7
                ds.SeriesNumber = 50
                ds.add_new(0x00280006, 'US', 0)
                ds.is_little_endian = True
                ds.fix_meta_info()

                # save pixel data and dicom file
                ds.PixelData = img_bbox.tobytes()
                ds.save_as('dicom_nodules/'+str(count)+'.dcm')
                ds = dicom.read_file('dicom_nodules/'+str(count)+'.dcm')
                pixels = get_pixels_hu([ds])
                image = pixels
                img = image[0]
    else:
        img_bbox = Image.fromarray((255*normalized_image).astype('uint8'))
        img_bbox = img_bbox.convert('RGB')
        img_bbox = np.asarray(img_bbox)

        # modify DICOM tags
        ds.PhotometricInterpretation = 'RGB'
        ds.SamplesPerPixel = 3
        ds.BitsAllocated = 8
        ds.BitsStored = 8
        ds.HighBit = 7
        ds.SeriesNumber = 50
        ds.add_new(0x00280006, 'US', 0)
        ds.is_little_endian = True
        ds.fix_meta_info()

        # save pixel data and dicom file
        ds.PixelData = img_bbox.tobytes()
        ds.save_as('dicom_nodules/'+str(count)+'.dcm')

In [ ]:
top, left, bottom, right = [211,99,291,158]
thickness = 4
img_bbox = Image.fromarray((255*display_img).astype('uint8'))
img_bbox = img_bbox.convert('RGB')
draw = ImageDraw.Draw(img_bbox)
for i in range(thickness):
    draw.rectangle(
        [left + i, top + i, right - i, bottom - i],
        outline=(255,0,0)
    )
del draw
img_bbox = np.asarray(img_bbox)

# modify DICOM tags
ds.PhotometricInterpretation = 'RGB'
ds.SamplesPerPixel = 3
ds.BitsAllocated = 8
ds.BitsStored = 8
ds.HighBit = 7
ds.add_new(0x00280006, 'US', 0)
ds.is_little_endian = True
ds.fix_meta_info()

# save pixel data and dicom file
ds.PixelData = img_bbox.tobytes()
ds.save_as('test.dcm')